TO DO
- Migrate tokens and keys
- Transform and upload to Blob Storage
- (Extended) Fetch reviews from letterboxd

In [0]:
%pip install requests
%pip install bs4

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

In [0]:
import requests
import json
from bs4 import BeautifulSoup
from random import uniform
import time

In [0]:
help(BeautifulSoup.find_all)

Help on function find_all in module bs4.element:

find_all(self, name=None, attrs={}, recursive=True, string=None, limit=None, **kwargs)
    Look in the children of this PageElement and find all
    PageElements that match the given criteria.
    
    All find_* methods take a common set of arguments. See the online
    documentation for detailed explanations.
    
    :param name: A filter on tag name.
    :param attrs: A dictionary of filters on attribute values.
    :param recursive: If this is True, find_all() will perform a
        recursive search of this PageElement's children. Otherwise,
        only the direct children will be considered.
    :param limit: Stop looking after finding this many results.
    :kwargs: A dictionary of filters on attribute values.
    :return: A ResultSet of PageElements.
    :rtype: bs4.element.ResultSet



In [0]:
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer KEY"
}

In [0]:
def get_movie_info(header, movie_id):
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?append_to_response=images%2Ccredits%2Ckeywords&language=en-US"
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer KEY"
    }
    details = {}
    letterboxd_url = ""
    tmdb_url = ""
    ltb_ag_rating = {
        "best_rating": 0,
        "worst_rating": 0,
        "review_count": 0,
        "rating_count": 0,
        "rating_value": 0
    } 
    filmstat_watches = 0
    filmstat_lists = 0
    filmstat_likes = 0
    film_slug = ""

    # Get movie information from TMDB
    try:
        # Get the movie details from TMDB
        response = requests.get(url, headers=headers)
        if (response.status_code != 200):
            raise Exception(f"Failed to fetch movie information from TMDB: {url}")
        result = json.loads(response.text)
        movie_imdb_id = result["imdb_id"]
        details = result
        print("Fetched successfully movie's TMDB information.")

        tmdb_url = f"https://www.themoviedb.org/movie/{movie_id}/"
        letterboxd_redirect_url = f"https://letterboxd.com/tmdb/{movie_id}"

        # Get movie reception from Letterboxd (FUTURE PLAN: Crawl reviews)
        res = requests.get(letterboxd_redirect_url)
        time.sleep(uniform(0, 1))
        if (res.status_code == 200):
            soup = BeautifulSoup(res.text, 'html.parser')

            film_slug = soup.select("[data-film-slug]")
            if (film_slug != None):
                film_slug_value = film_slug[0].get('data-film-slug')
                print("Fetched successfully movie's Letterboxd film slug.")
            else:
                raise Exception('No Letterboxd film slug found.')

            info_json = soup.find('script', type='application/ld+json')
            if (info_json):
                info_text = info_json.string.strip()
                cleaned_info_text = info_text.replace("/* <![CDATA[ */", "").replace("/* ]]> */", "")
                info = json.loads(cleaned_info_text)
                print("Fetched successfully movie's Letterboxd info json.")
                if ("@id" in info):
                    letterboxd_url = info["@id"]
                else:
                    print("Letterboxd url not found, which is weird.")    
                if ("aggregateRating" in info):
                    ag_rating = info["aggregateRating"]
                    ltb_ag_rating["best_rating"] = ag_rating["bestRating"]
                    ltb_ag_rating["worst_rating"] = ag_rating["worstRating"]
                    ltb_ag_rating["review_count"] = ag_rating["reviewCount"]
                    ltb_ag_rating["rating_count"] = ag_rating["ratingCount"]
                    ltb_ag_rating["rating_value"] = ag_rating["ratingValue"]
                else:
                    print('Letterboxd aggregate rating information not found.')                                    
            else:
                print('Letterboxd info json not found.')

            # Get stats of film
            film_stats_url = f"https://letterboxd.com/csi/film/{film_slug_value}/stats/"
            film_stats_response = requests.get(film_stats_url)
            time.sleep(uniform(0, 1))
            if (film_stats_response and film_stats_response.status_code == 200):
                film_stats_element = BeautifulSoup(film_stats_response.text, 'html.parser')
                if (film_stats_element != None):
                    # filmstat watches
                    filmstat_watches = film_stats_element.find('li', class_='stat filmstat-watches')
                    filmstat_watches_a_element = filmstat_watches.find('a') if filmstat_watches != None else None
                    filmstat_watches_value = filmstat_watches_a_element.get('title') if filmstat_watches != None else 0

                    # film stat lists
                    filmstat_lists = film_stats_element.find('li', class_='stat filmstat-lists')
                    filmstat_lists_a_element = filmstat_lists.find('a') if filmstat_lists != None else None
                    filmstat_lists_value = filmstat_lists_a_element.get('title') if filmstat_lists != None else 0

                    # film stat likes
                    filmstat_likes = film_stats_element.find('li', class_='stat filmstat-likes')
                    filmstat_likes_a_element = filmstat_likes.find('a') if filmstat_likes != None else None
                    filmstat_likes_value = filmstat_likes_a_element.get('title') if filmstat_likes != None else 0
                    print("Fetched successfully movie's Letterboxd film stats.")
            else:
                print("Letterboxd film stats not found.")
        else:
            print("Letterboxd movie page not found.")

        movie_info = {
            **details,
            "ltb_film_slug": film_slug_value,
            "filmstat_watches": filmstat_watches_value,
            "filmstat_lists": filmstat_lists_value, 
            "filmstat_likes": filmstat_likes_value,
            "letterboxd_ag_rating": ltb_ag_rating
        }
        return movie_info
    except Exception as e:
        print(f"Error getting movie details from TMDB: {e})")

In [0]:
print("hi")

hi


In [0]:
get_movie_info(headers, 533535)

Fetched successfully movie's TMDB information.
Fetched successfully movie's Letterboxd film slug.
Fetched successfully movie's Letterboxd info json.
Fetched successfully movie's Letterboxd film stats.


{'adult': False,
 'backdrop_path': '/9l1eZiJHmhr5jIlthMdJN5WYoff.jpg',
 'belongs_to_collection': {'id': 448150,
  'name': 'Deadpool Collection',
  'poster_path': '/30c5jO7YEXuF8KiWXLg9m28GWDA.jpg',
  'backdrop_path': '/hBQOWY8qWXJVFAc8yLTh1teIu43.jpg'},
 'budget': 200000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 35, 'name': 'Comedy'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'https://www.marvel.com/movies/deadpool-and-wolverine',
 'id': 533535,
 'imdb_id': 'tt6263850',
 'origin_country': ['US'],
 'original_language': 'en',
 'original_title': 'Deadpool & Wolverine',
 'overview': 'A listless Wade Wilson toils away in civilian life with his days as the morally flexible mercenary, Deadpool, behind him. But when his homeworld faces an existential threat, Wade must reluctantly suit-up again with an even more reluctant Wolverine.',
 'popularity': 2838.675,
 'poster_path': '/8cdWjvZQUExUUTzyp4t6EDMubfO.jpg',
 'production_companies': [{'id': 420,
   'logo_path': '/hU

In [0]:
# Get movies released in 01-08/2024
movies_info = []
discover_url = "https://api.themoviedb.org/3/discover/movie?include_adult=false&include_video=false&language=en-US&page={page}&primary_release_date.gte=2024-01-01&primary_release_date.lte=2024-08-31&sort_by=popularity.desc"

response = requests.get(discover_url.format(page=500), headers=headers)
if (response.status_code == 200):
    result = json.loads(response.text)
    npage = 10
    # npage = result["total_pages"]
    nresult = result["total_results"]
    print(f"Total pages: {npage}, Total results: {nresult}")
    for page in range(1, npage + 1):
        print(f"Fetching page {page}")
        search_res = requests.get(discover_url.format(page=page), headers=headers)
        # when mass crawling remember to change the sort into primary_release_date.desc
        if (search_res.status_code == 200):
            search_result = json.loads(search_res.text)
            for result in search_result["results"]:
                try:
                    movie_id = result["id"]
                    title = result["title"]
                    print(f"Fetching movie {title} with id {movie_id}...")
                    movie_info = get_movie_info(headers, movie_id)
                    movies_info.append(movie_info)
                except Exception as e:
                    print(e)
            time.sleep(uniform(0, 1))

Total pages: 10, Total results: 25461
Fetching page 1
Fetching movie Deadpool & Wolverine with id 533535...
Fetched successfully movie's TMDB information.
Fetched successfully movie's Letterboxd film slug.
Fetched successfully movie's Letterboxd info json.
Fetched successfully movie's Letterboxd film stats.
Fetching movie Borderlands with id 365177...
Fetched successfully movie's TMDB information.
Fetched successfully movie's Letterboxd film slug.
Fetched successfully movie's Letterboxd info json.
Fetched successfully movie's Letterboxd film stats.
Fetching movie Inside Out 2 with id 1022789...
Fetched successfully movie's TMDB information.
Fetched successfully movie's Letterboxd film slug.
Fetched successfully movie's Letterboxd info json.
Fetched successfully movie's Letterboxd film stats.
Fetching movie Rebel Ridge with id 646097...
Fetched successfully movie's TMDB information.
Fetched successfully movie's Letterboxd film slug.
Fetched successfully movie's Letterboxd info json.
Fet

In [0]:
filename = 'movies.json'
with open(filename, 'w') as json_file:
    json.dump(movies_info, json_file)

In [0]:
people = {}

In [0]:
# Initialize a counter and a start time
request_counter = 0
start_time = time.time()

def get_person_details(headers, person_id):
    global request_counter, start_time
    
    # Check if we have made 40 requests
    if request_counter >= uniform(30, 40):
        # Calculate how much time has passed since we started making requests
        elapsed_time = time.time() - start_time
        if elapsed_time < 1:
            # If it's been less than a second, sleep for the remainder of the second
            time.sleep(1 - elapsed_time)
        # Reset the counter and start time
        request_counter = 0
        start_time = time.time()
    
    person_details_url = f"https://api.themoviedb.org/3/person/{person_id}?language=en-US"
    person_details_res = requests.get(person_details_url, headers=headers)
    request_counter += 1  # Increment the request counter after each request
    
    if person_details_res.status_code == 200:
        return person_details_res.json()

In [0]:
get_person_details(headers, 1)

{'adult': False,
 'also_known_as': ['George Walton Lucas Jr. ',
  '乔治·卢卡斯',
  'Джордж Лукас',
  'ジョージ・ルーカス',
  'จอร์จ ลูคัส',
  '조지 루카스',
  'جورج لوكاس'],
 'biography': "George Walton Lucas Jr. (born May 14, 1944) is an American filmmaker and entrepreneur. Lucas is known for creating the Star Wars and Indiana Jones franchises and founding Lucasfilm, LucasArts and Industrial Light & Magic. He served as chairman of Lucasfilm before selling it to The Walt Disney Company in 2012.\n\nAfter graduating from the University of Southern California in 1967, Lucas co-founded American Zoetrope with filmmaker Francis Ford Coppola. Lucas wrote and directed THX 1138 (1971), based on his earlier student short Electronic Labyrinth: THX 1138 4EB, which was a critical success but a financial failure. His next work as a writer-director was the film American Graffiti (1973), inspired by his youth in early 1960s Modesto, California, and produced through the newly founded Lucasfilm. The film was critically an

In [0]:
len(movies_info)

200

In [0]:
i = 200*300/40/60

In [0]:
i

25.0

In [0]:
people = {}

In [0]:
pos = 0
for movie in movies_info:
    movie_id = movie['id']
    credits = movie['credits']
    if credits is not None:
        print(f"Fetching person details for credits in movie {movie_id} at position {pos}...")
        pos += 1
        cast = credits["cast"]
        for person in cast:
            person_id = person["id"]
            if person_id not in people:
                person_details = get_person_details(headers, person_id)
                people[person_id] = person_details
        crew = credits["crew"]
        for person in crew:
            person_id = person["id"]
            if person_id not in people:
                person_details = get_person_details(headers, person_id)
                people[person_id] = person_details
        with open(f"people/{movie_id}_credits.json", "w") as f:
            json.dump(people, f)
            people = {}

Fetching person details for credits in movie 533535 at position 0...
Fetching person details for credits in movie 365177 at position 1...
Fetching person details for credits in movie 1022789 at position 2...
Fetching person details for credits in movie 646097 at position 3...
Fetching person details for credits in movie 519182 at position 4...
Fetching person details for credits in movie 573435 at position 5...
Fetching person details for credits in movie 970347 at position 6...
Fetching person details for credits in movie 718821 at position 7...
Fetching person details for credits in movie 923667 at position 8...
Fetching person details for credits in movie 1079091 at position 9...
Fetching person details for credits in movie 945961 at position 10...
Fetching person details for credits in movie 831815 at position 11...
Fetching person details for credits in movie 762441 at position 12...
Fetching person details for credits in movie 1160018 at position 13...
Fetching person details for